Code to check the mean and variance of the recovered estimates of the autocovariance of $R$ from a simulated $R(t)$.

In [ ]:
import numpy as np
from matplotlib import animation, pyplot as plt
import seaborn as sns
import pandas as pd`
import scipy.linalg as linalg
from scipy.optimize import minimize 

import warnings
from scipy.linalg import toeplitz
warnings.filterwarnings('ignore')
%pylab inline 

In [ ]:
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 22}

axes = {'titlesize'  : 22,
        'labelsize'  : 22}

legend = {'fontsize'  : 22}

figure = {'figsize'  : (10,5)}

matplotlib.rc('font', **font)
matplotlib.rc('axes', **axes)
matplotlib.rc('legend', **legend)
matplotlib.rc('figure', **figure)

In [ ]:
from simulate_activity import *
from est_param import *

In [ ]:
trials = 100
est_rho = np.empty((trials,))
est_l = np.empty((trials,))
est_p = np.empty((trials,))
est_sig_r = np.empty((trials,))

est_rho_MSE = np.empty((trials,))
est_l_MSE = np.empty((trials,))
est_p_MSE = np.empty((trials,))
est_sig_r_MSE = np.empty((trials,))

RHO, L, P = 5,8,1.1
T, LAG = 500,50
MU_M, ALPHA = 10,.9
SIG_R, SIG_G = 2,.25
MY_Y,SIG_Y2, L_Y = 1,.1,2

s = SimulateWorm(Rho = RHO,L = L, P = P, T = T, Lag = LAG, Mu_m= MU_M, Sig_r=SIG_R,alpha = ALPHA)

for i in range(trials):
    if i%10 == 0:
        print(i)
    
    R_t = s.gen_R_t('matrix')
    R_t = R_t - np.mean(R_t)
    (xx,R_t_corr) = calc_corr(R_t, R_t,LAG)
    inf_params = est_MSE_R(xx,R_t_corr)
    
    # save params
    est_rho_MSE[i] = inf_params[0]
    est_l_MSE[i] = inf_params[1]
    est_p_MSE[i] = inf_params[2]
    est_sig_r_MSE[i] = inf_params[3]
    
    #jiggled_params = np.array([RHO, L, P, SIG_R]) + np.random.normal(4)*.25
    mle_res = est_MLE_R(R_t, inf_params,toPrint=False)
    # save params
    est_rho[i] = mle_res[0]
    est_l[i] = mle_res[1]
    est_p[i] = mle_res[2]
    est_sig_r[i] = mle_res[3]

In [ ]:
std(est_rho)

In [ ]:
print("--- MLE ---")
print("STD:")
print("Rho %1.3f\tL: %1.3f\tP: %1.3f\tSigma_r: %1.3f\t"
      % (std(est_rho), std(est_l), std(est_p), std(est_sig_r)))
print("Medians:")
print("Rho %1.3f\tL: %1.3f\tP: %1.3f\tSigma_r: %1.3f\t"
      % (median(est_rho), median(est_l), median(est_p), median(est_sig_r)))
print("--- MSE ---")
print("STD:")
print("Rho %1.3f\tL: %1.3f\tP: %1.3f\tSigma_r: %1.3f\t"
      % (std(est_rho_MSE), std(est_l_MSE), std(est_p_MSE), std(est_sig_r_MSE)))
print("Medians:")
print("Rho %1.3f\tL: %1.3f\tP: %1.3f\tSigma_r: %1.3f\t"
      % (median(est_rho_MSE), median(est_l_MSE), median(est_p_MSE), median(est_sig_r_MSE)))

In [ ]:
# Evaluate how well this did:
true_r = create_fitted(xx,[RHO, L, P, SIG_R])
fitted_r = create_fitted(xx,[median(est_rho), median(est_l), median(est_p), median(est_sig_r)])
plt.plot(xx,true_r,label='True')
plt.plot(xx,fitted_r,label='Fitted')

plt.legend()


In [ ]:
f,ax = plt.subplots(1,4,figsize=(20,5))
ax[0].hist(est_rho);
ax[0].set_title('Values of Rho')
ax[1].hist(est_l);
ax[1].set_title('Values of L')
ax[2].hist(est_p);
ax[2].set_title('Values of p')
ax[3].hist(est_sig_r);
ax[3].set_title('Values of sig_r')